# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# output data cities

output_cities_file = "cities.csv"

Results_part1_df = pd.read_csv(output_cities_file)

Results_part1_df.head()


,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,trairi,298.52,87,-39.2689,-3.2778,90,BR,4.20,1613102612
1,clarence town,307.04,51,151.7833,-32.5833,0,AU,3.09,1613102612
2,hilo,298.15,61,-155.0900,19.7297,90,US,3.60,1613102537
3,padang,304.15,66,100.3543,-0.9492,20,ID,2.57,1613102606
4,butaritari,300.55,76,172.7902,3.0707,0,KI,5.28,1613102612


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [10]:
locations = Results_part1_df[["Latitude", "Longitude"]]
humidity = Results_part1_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
ideal_df = Results_part1_df[(Results_part1_df['Temperature'] >= 200) & (Results_part1_df['Temperature'] <= 250)]
ideal_df = ideal_df[ideal_df['Windspeed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df.head()

,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
95,haines junction,243.15,69,-137.5108,60.7522,0,CA,0.51,1613102626
493,glendive,244.15,63,-104.7125,47.1053,1,US,3.60,1613102694
537,whitehorse,249.82,64,-135.0538,60.7161,1,CA,7.72,1613102701


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date,Hotel Name
95,haines junction,243.15,69,-137.5108,60.7522,0,CA,0.51,1613102626,
493,glendive,244.15,63,-104.7125,47.1053,1,US,3.60,1613102694,
537,whitehorse,249.82,64,-135.0538,60.7161,1,CA,7.72,1613102701,


In [21]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date,Hotel Name
95,haines junction,243.15,69,-137.5108,60.7522,0,CA,0.51,1613102626,Kluane Green Sprout
493,glendive,244.15,63,-104.7125,47.1053,1,US,3.60,1613102694,Holiday Inn Express & Suites Glendive
537,whitehorse,249.82,64,-135.0538,60.7161,1,CA,7.72,1613102701,Raven Inn Whitehorse


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))